In [1]:
import os,re
from pyspark.sql.functions import col, array_contains, array_position,year,to_date

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1653719621549_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1653719621549_0001,pyspark,idle,Link,Link,✔


In [3]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1653719621549_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1653719621549_0002,pyspark,idle,Link,Link,✔


#### Installing Packages

In [8]:
#sc.uninstall_package('pandas')
sc.install_pypi_package("time")
sc.install_pypi_package("cython")
sc.install_pypi_package("boto3")
sc.install_pypi_package("s3fs")
sc.install_pypi_package("pandas") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…




  Running setup.py bdist_wheel for aiobotocore: started
  Running setup.py bdist_wheel for aiobotocore: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/96/9e/df/18627cbd5cdcad23931cca4784d516adf093c93f30aeb14f26
  Running setup.py bdist_wheel for multidict: started
  Running setup.py bdist_wheel for multidict: finished with status 'error'
  Complete output from command /tmp/1653740622167-0/bin/python -u -c "import setuptools, tokenize;__file__='/mnt/tmp/pip-build-a7eepjgr/multidict/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpee2nzmtqpip-wheel- --python-tag cp37:
  *********************
  * Accelerated build *
  *********************
  /usr/lib64/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'project_urls'
    warnings.warn(msg)
  running bdist_wheel
  running build
  running build_py
  creating build
  c

In [ ]:
sc.list_packages()

#### Demographic Data

In [2]:
df_demographic=spark.read.option("header",True).csv("s3://pas-biogen-sagemaker/nlp/uh/onetime-historical/04-27-2021/demographics.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
### Remove duplicate values
print(df_demographic.count())
df_demographic = df_demographic.dropDuplicates(['patient_key'])
print(df_demographic.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

310824
['patient_key', 'pat_last_name', 'pat_first_name', 'pat_mrn', 'pat_dob', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_country', 'pat_marital_status', 'pat_smoking_status', 'pat_status']

#### Anchor Features

In [4]:
df_anchor=spark.read.option("header",True).csv("s3://pas-biogen-sagemaker/nlp/Anchor_roll_up_new.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_anchor = df_anchor.drop('Anchors')
df_anchor.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment']

In [17]:
lst_Thought_Disturbance = list(df_anchor.select('Thought_Disturbance').toPandas()['Thought_Disturbance'])
lst_Abnormal_Behavior = list(df_anchor.select('Abnormal_Behavior').toPandas()['Abnormal_Behavior'])
lst_Suicidal_Intent = list(df_anchor.select('Suicidal_Intent').toPandas()['Suicidal_Intent'])
lst_Disorientated = list(df_anchor.select('Disorientated').toPandas()['Disorientated'])
lst_Mental_Health_Impairment = list(df_anchor.select('Mental_Health_Impairment').toPandas()['Mental_Health_Impairment'])
lst_Memory_Loss = list(df_anchor.select('Memory_Loss').toPandas()['Memory_Loss'])
lst_Clouded_Consciousness = list(df_anchor.select('Clouded_Consciousness').toPandas()['Clouded_Consciousness'])
lst_Agitation = list(df_anchor.select('Agitation').toPandas()['Agitation'])
lst_Congnitive_Impairment = list(df_anchor.select('Congnitive_Impairment').toPandas()['Congnitive_Impairment'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2086, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



In [10]:
# List of CUI, ICD10, Snomed for Alzheimer Disease
lst_cui = ['C0002395',
 'C0236640',
 'C0236644',
 'C0236645',
 'C0236646',
 'C0236647',
 'C0236648',
 'C0236649',
 'C0333736',
 'C0338445',
 'C0338446',
 'C0338449',
 'C0338450',
 'C0338458',
 'C0338459',
 'C0349078',
 'C0424954',
 'C0450989',
 'C0477364',
 'C0494463',
 'C0750901',
 'C1271045',
 'C1863051',
 'C3241944',
 'C3494731',
 'C3662845',
 'C3665464',
 'C3697680',
 'C3697863',
 'C3697898',
 'C3698264',
 'C3698286',
 'C3698287',
 'C4075209',
 'C4075348',
 'C4273575',
 'C4302404',
 'C4303482',
 'C4518848',
 'C0085400',
 'C0752347',
 'C2316550',
 'C2316551',
 'C3532942']

lst_icd10 = ['G30',
 'G30.9',
 'F00.2',
 'G30.8',
 'G30.1',
 'F00.1',
 'G30.0',
 'F00.0',
 'F00',
 'F00.9']

lst_snomed = ['26929004',
 '66108005',
 '65096006',
 '54502004',
 '10532003',
 '55009008',
 '26852004',
 '4817008',
 '29209006',
 '230265002',
 '230266001',
 '230268000',
 '230269008',
 '230279005',
 '230280008',
 '161108005',
 '273269001',
 '416975007',
 '416780008',
 '394877006',
 '230267005',
 '6475002',
 '429161000124103',
 '1581000119101',
 '105421000119105',
 '698955006',
 '698954005',
 '97751000119108',
 '142001000119106',
 '141991000119109',
 '714359006',
 '714360001',
 '142011000119109',
 '722600006',
 '721219005',
 '16219201000119101',
 '85775002',
 '80098002',
 '431531007',
 '432954006',
 '79341000119107']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# ALzheimer CUI, ICD10 and SNOMED
columns = ['lst_cui']
df_lst_cui = spark.createDataFrame(zip(lst_cui), columns)
columns = ['lst_icd10']
df_lst_icd10 = spark.createDataFrame(zip(lst_icd10), columns)
columns = ['lst_snomed']
df_lst_snomed = spark.createDataFrame(zip(lst_snomed), columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_lst_cui.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
| lst_cui|
+--------+
|C0002395|
|C0236640|
|C0236644|
|C0236645|
|C0236646|
+--------+
only showing top 5 rows

In [13]:
import boto3
#s3://pas-biogen-sagemaker/Antony/Preprocessing/Ongoing/
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('pas-biogen-sagemaker')
chunks=[]
for object_summary in my_bucket.objects.filter(Prefix="nlp/uh/onetime-historical/04-27-2021/raw_notes/results_UMLS/CSV/"):
    if '.parquet' in str(object_summary.key):
        chunks.append(object_summary.key)  


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
len(chunks)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11

In [15]:
import s3fs
import pandas as pd
import time
from pyspark.sql.functions import arrays_overlap, array
from pyspark.sql import udf,DataFrame
import pyspark.sql.functions as F
import functools
import sys

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 's3fs'
Traceback (most recent call last):
ModuleNotFoundError: No module named 's3fs'



In [51]:
def my_print(text):
    sys.stdout.write(str(text))
    sys.stdout.flush()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
s3 = s3fs.S3FileSystem(anon=False)
count = 0
output_dfs = []

for chunk in chunks:
    start_time = time.time()
    count += 1
    df_final = spark.read.parquet('s3://pas-biogen-sagemaker/'+ chunk)
    
    #text = 'Size of Data '+ str(count) + ' - '+ str(df_final.count())
    #my_print(text)
    
    df_final =df_final.withColumnRenamed('Concept_ID','UMLS_CUI')

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Thought_Disturbance'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Thought_Disturbance', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Abnormal_Behavior'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Abnormal_Behavior', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Suicidal_Intent'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Suicidal_Intent', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Disorientated'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Disorientated', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Mental_Health_Impairment'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Mental_Health_Impairment', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Memory_Loss'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Memory_Loss', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Clouded_Consciousness'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Clouded_Consciousness', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Agitation'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Agitation', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Congnitive_Impairment'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Congnitive_Impairment', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    df_anchor2 = df_lst_cui.withColumn('UMLS_CUI',df_lst_cui['lst_cui'])
    df_anchor2 = df_anchor2[['UMLS_CUI']]
    df_final = df_final.join(df_anchor2.withColumn('Alzheimer', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

    output_dfs.append(df_final)
    df_final2 = functools.reduce(DataFrame.union, output_dfs)
    
    df_final2 = df_final2.withColumn("patient_key",df_final2.patient_key.cast('int'))
    df_demographic = df_demographic.withColumn("patient_key",df_demographic.patient_key.cast('int'))
    
    df_final3 = df_final2.join(df_demographic,['patient_key'],how='inner')  
    df_final3 = df_final3.dropDuplicates()

df_final3.coalesce(1).write.parquet("s3://pas-biogen-sagemaker/Antony/Ongoing_Output/2021_historical")

#time_taken = ((time.time() - start_time) / 60)
#text = str(count) + "- " + "Time Taken - " + str(time_taken) + " min"
#my_print(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
time_taken = ((time.time() - start_time) / 60)
text = 'Size of Data '+ str(count) + ' - '+ str(df_final.count())
my_print(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Size of Data 1 - 31540279

In [36]:
df_final2 = df_final2.withColumn("patient_key",df_final2.patient_key.cast('int'))
df_demographic = df_demographic.withColumn("patient_key",df_demographic.patient_key.cast('int'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_demographic.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patient_key: integer (nullable = true)
 |-- pat_last_name: string (nullable = true)
 |-- pat_first_name: string (nullable = true)
 |-- pat_mrn: string (nullable = true)
 |-- pat_dob: string (nullable = true)
 |-- pat_gender: string (nullable = true)
 |-- pat_race: string (nullable = true)
 |-- pat_ethinicity: string (nullable = true)
 |-- pat_city: string (nullable = true)
 |-- pat_zip: string (nullable = true)
 |-- pat_county: string (nullable = true)
 |-- pat_state: string (nullable = true)
 |-- pat_country: string (nullable = true)
 |-- pat_marital_status: string (nullable = true)
 |-- pat_smoking_status: string (nullable = true)
 |-- pat_status: string (nullable = true)

In [35]:
df_final2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UMLS_CUI: string (nullable = true)
 |-- patient_key: integer (nullable = true)
 |-- provider_id: string (nullable = true)
 |-- facility_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- negex: boolean (nullable = false)
 |-- span: string (nullable = true)
 |-- UMLS_Name: string (nullable = true)
 |-- TUIs: string (nullable = true)
 |-- Child_semantic_label: string (nullable = true)
 |-- Parent_semantic_label: string (nullable = true)
 |-- Similarity_Score: double (nullable = true)
 |-- Thought_Disturbance: boolean (nullable = false)
 |-- Abnormal_Behavior: boolean (nullable = false)
 |-- Suicidal_Intent: boolean (nullable = false)
 |-- Disorientated: boolean (nullable = false)
 |-- Mental_Health_Impairment: boolean (nullable = false)
 |-- Memory_Loss: boolean (nullable = false)
 |-- Clouded_Consciousness: boolean (nullable = false)
 |-- Agitation: boolean (nullable = false)
 |-- Congnitive_Impairment: boolean (nullable 

## ROugh Work

In [18]:
df_final.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

31540279

In [42]:
import sys

def my_print(text):
    sys.stdout.write(str(text))
    sys.stdout.flush()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
text = "hello this is printed"
my_print(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hello this is printed

In [41]:
df_final.coalesce(1).write.parquet("s3://pas-biogen-sagemaker/Antony/Ongoing_Output/2019_historical")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pyspark.sql.functions as F

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Thought_Disturbance'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Thought_Disturbance', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Abnormal_Behavior'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Abnormal_Behavior', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Suicidal_Intent'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Suicidal_Intent', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Disorientated'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Disorientated', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Mental_Health_Impairment'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Mental_Health_Impairment', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Memory_Loss'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Memory_Loss', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Clouded_Consciousness'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Clouded_Consciousness', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Agitation'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Agitation', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_anchor.withColumn('UMLS_CUI',df_anchor['Congnitive_Impairment'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Congnitive_Impairment', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

df_anchor2 = df_lst_cui.withColumn('UMLS_CUI',df_lst_cui['lst_cui'])
df_anchor2 = df_anchor2[['UMLS_CUI']]
df_final = df_final.join(df_anchor2.withColumn('Alzheimer', F.lit(True)), 'UMLS_CUI', 'left').fillna(False)

In [ ]:
print(df_final[df_final['Alzheimer'] == 1].distinct().count())

In [38]:
output = df_final2.join(df_demographic,['patient_key'],how='inner')  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
output[['patient_key','UMLS_CUI','provider_id']].show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-----------+
|patient_key|UMLS_CUI|provider_id|
+-----------+--------+-----------+
|     800966|C1704653| 1033124813|
|    1639971|C0013770| SRN0050793|
|    1385981|C0024779| 1053367250|
|    1385981|C0024779| 1053367250|
|    3025611|C3890564| SRN0006317|
|   72124940|C3899513| SRN0030667|
|   72285645|C2063616| 1740414457|
|   72285645|C2063616| 1740414457|
|   72285645|C2063616| 1740414457|
|   72285645|C2063616| 1740414457|
|   72377207|C0024779| SRN0040897|
|   72377207|C0024779| SRN0040897|
|   72377207|C0024779| SRN0040897|
|   72377207|C3814778| SRN0040897|
|    2161568|C3543515| SRN0023779|
|    7659660|C0007012| SRN0014151|
|   72020812|C0024779| 1568466290|
|   72020812|C0024779| 1568466290|
|    8026121|C0402015| SRN0047207|
|    3145184|C3543515| 1962516690|
+-----------+--------+-----------+
only showing top 20 rows

In [25]:
output.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['patient_key', 'UMLS_CUI', 'provider_id', 'facility_id', 'text', 'label', 'negex', 'span', 'UMLS_Name', 'TUIs', 'Child_semantic_label', 'Parent_semantic_label', 'Similarity_Score', 'Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment', 'Alzheimer', 'pat_last_name', 'pat_first_name', 'pat_mrn', 'pat_dob', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_country', 'pat_marital_status', 'pat_smoking_status', 'pat_status']

In [34]:
op2 = df_final[df_final['Alzheimer'] == 0]
op2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

31539959

In [35]:
print(op2[op2['Congnitive_Impairment']==1].count())
print(op2[op2['Thought_Disturbance']==1].count())
print(op2[op2['Abnormal_Behavior']==1].count())
print(op2[op2['Suicidal_Intent']==1].count())
print(op2[op2['Disorientated']==1].count())
print(op2[op2['Mental_Health_Impairment']==1].count())
print(op2[op2['Memory_Loss']==1].count())
print(op2[op2['Clouded_Consciousness']==1].count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5479
1411
3688
3595
75
8451
3077
2131